In [13]:
import pandas as pd
import osmnx as ox
import cenpy
import os
import geopandas as gpd

print(os.getcwd())


/home/jovyan/work/business-opportunity-graph/notebooks


In [5]:
acs = cenpy.products.ACS()

In [11]:
SD_crs = 'EPSG:26946'
common_crs = "EPSG:4326"


data_path = '../data/'

In [14]:
cities_df = pd.read_json("../data/city.json")
# cities_df.head()

boundaries_zip = 'City_and_County_Boundaries.zip'
boundaries_path = os.path.join(data_path, boundaries_zip)
boundaries_gdf = gpd.read_file(boundaries_path)
boundaries_gdf = boundaries_gdf.to_crs(common_crs)

sd_boundaries_gdf = boundaries_gdf[boundaries_gdf['COUNTY_NAM'] == 'San Diego']
sd_boundaries_mixed = sd_boundaries_gdf[['CITY','geometry']]

sd_cities_boundaries_gdf= sd_boundaries_mixed.rename(columns={"CITY": "City"}).dissolve(by= 'City').reset_index()


In [19]:
cities_names_list = cities_df['name'].unique()

attribute_variables = ['B03002_001E','B09019_001E']
cities_attributes_data = [['City', "Area", "Total Population", 'Intersection Count', 'Street Density', 'Total Households']]

for city in cities_names_list:
  city_data =[]
  place_name = city + ', CA'
  city_gdf = ox.geocode_to_gdf(place_name) #from gemini
  city_geometry = city_gdf.geometry.iloc[0] #from gemini
  city_area = sd_cities_boundaries_gdf[sd_cities_boundaries_gdf['City']==city].to_crs('EPSG:26946').area
  # city_area_m2 = ox.project_gdf(city_gdf).area.iloc[0] # Area in square meters #from gemini
  # city_area = city_area_m2 / (1000 * 1000) # Convert to square kilometers for the area variable if needed #from gemini

  # city_pop_gdf = acs.from_place(place_name, variables=attribute_variables)
  # city_pop_tot = city_pop_gdf['B03002_001E'].sum()
  # city_tot_household = city_pop_gdf['B09019_001E'].sum()
  G_city_drive = ox.graph.graph_from_place(place_name, network_type="drive")
  city_street_stats = ox.stats.basic_stats(G_city_drive, area=city_area)
  city_intersection_count = city_street_stats['intersection_count']
  city_street_density_km =city_street_stats['street_density_km']
  city_data = [city,city_area, city_pop_tot, city_intersection_count, city_street_density_km, city_tot_household]
  cities_attributes_data.append(city_data)


NameError: name 'city_pop_tot' is not defined

In [23]:
# attribute_variables
city_pop_gdf = acs.from_place(place_name, variables=attribute_variables[0])

Matched: La Mesa, CA to La Mesa city within layer Incorporated Places


TypeError: sjoin() got an unexpected keyword argument 'op'